In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

from tqdm import tqdm

from sklearn.model_selection import train_test_split

import re
import spacy
from spacy.tokens import DocBin

In [2]:
def json_tweets_to_pd(jsonfile):
    '''
    read paginated json file to df
    input (string): .json file created by searchtweets module
    output (pd.DataFrame)
    '''
    raw_df = pd.read_json(jsonfile, lines=True)
    df = pd.DataFrame()
    for page in raw_df['data']:
        df = pd.concat([df, pd.DataFrame(page)])
    return df

# Creating training set

In [3]:
df_positive = json_tweets_to_pd('get_tweets/tweets_positive.json')

# drop tweets withheld by the twitter accounts
df_positive = df_positive.drop(df_positive[df_positive.withheld.notnull()].index)

df_positive.drop(columns=['id', 'withheld'], inplace=True)
df_positive = df_positive.astype({'text':'string'})
df_positive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15464 entries, 0 to 499
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    15464 non-null  string
dtypes: string(1)
memory usage: 241.6 KB


In [4]:
df_tweets = pd.read_csv('data/tweets_dow.csv', usecols = ['text'], dtype={'text':'string'})
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186204 entries, 0 to 186203
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    186204 non-null  string
dtypes: string(1)
memory usage: 1.4 MB


In [5]:
n_positive = len(df_positive)
df_positive['climate_related'] = np.ones(n_positive, dtype=bool)

df_negative = df_tweets.sample(n_positive, random_state = 2022)

In [6]:
# keywords = ["climate action", "environment", "climate change", "renewable", "recycle", "earth", "emission", "carbon footprint", ...]
with open("keywords.txt", "r") as f:
    keywords = [line.rstrip() for line in f]
negative = df_negative.text.apply(lambda text: not any([keyword in text.lower() for keyword in keywords]))

In [7]:
df_negative = df_negative[negative]
df_negative['climate_related'] = np.zeros(len(df_negative), dtype=bool)

In [8]:
df = pd.concat([df_positive, df_negative]).sample(frac=1)

# Preprocessing

In [9]:
def clean_up(tweet): 

    parsed_tweet = tweet
    
    # remove url
    parsed_tweet = re.sub(r"\S*https?:\S*", "", parsed_tweet, flags=re.MULTILINE)

    # remove non-ascii
    parsed_tweet = re.sub(r"[^\x00-\x7F]+", "", parsed_tweet)

    # remove 'RT'
    parsed_tweet = re.sub(r"^RT\s", "", parsed_tweet)
    return parsed_tweet

In [10]:
def preprocess(df, embed):
    '''
    Preprocess the dataframe into spacy pipeline for later classification
    ---
    Input:
    df (DataFrame): Pandas dataframe containing the raw text and outputs.
    embed (str): Name of pipeline embedding used

    Output:
    df (DataFrame): Preprocessed input dataframe
    docs (doc): SpaCy doc object that stores text data along with classification
    '''

    # clean up tweets
    df.text = df.text.apply(clean_up)

    # Store the data into tuples
    data = tuple(zip(df.text.tolist(), df.climate_related.tolist())) 
    
    # Load English library from SpaCy
    nlp = spacy.load(embed)
    print(data[0])

    # Storage for docs
    docs = []

    # One-hot encoding for the classifications
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        
        if label:
            doc.cats['climate_related'] = 1
        else:
            doc.cats['climate_related'] = 0
        # print(doc.cats)
        
        docs.append(doc)
    return df, docs

In [11]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(df, test_size=0.2)

print("Train:",len(train_df), "Valid:", len(valid_df))

Train: 24430 Valid: 6108


In [12]:
# make sure the model in installed in current conda env

# embed = "en_core_web_sm"
# embed = "en_core_web_md"
# embed = "en_core_web_lg"
embed = "en_core_web_trf"

assert spacy.util.is_package(embed), embed + " is not installed. To install, run: `!python -m spacy download \"" + embed + "\"`"

In [13]:
# Covert the train and valid dataframes to .spacy files for training

# Preprocess the dataframes for train data
train_data, train_docs = preprocess(train_df, embed)
# # Save data and docs in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./spacy_data/textcat_train.spacy")

# # Preprocess the dataframes for validation data
valid_data, valid_docs = preprocess(valid_df, embed)
# # Save data and docs in a binary file to disc
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("./spacy_data/textcat_valid.spacy")

('@aster_cloud: Want Faster Data And A Cleaner Planet? Start Mining Asteroids - \n\n#planet #Data #ClimateCrisis #Research #technology\n\nhttp', True)


  0%|          | 0/24430 [00:00<?, ?it/s]/Users/joe/miniconda3/envs/esg-frontier/lib/python3.10/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
100%|██████████| 24430/24430 [1:21:48<00:00,  4.98it/s]


("@Ym78200: Want to look at the world with fresh eyes?\n\nSo, let's meet at #Vivatech, 15-18 june,\non @3DEXPERIENCELab booth K52,\nand let's", True)


100%|██████████| 6108/6108 [44:38<00:00,  2.28it/s]  


In [20]:
!python -m spacy init config "./config.cfg" --lang en --pipeline textcat_multilabel --optimize efficiency --force

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: textcat_multilabel
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [21]:
!python -m spacy train ./config.cfg --output ./output --paths.train ./spacy_data/textcat_train.spacy --paths.dev ./spacy_data/textcat_valid.spacy --verbose

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[2022-06-15 19:17:46,125] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-06-15 19:17:46,667] [INFO] Set up nlp object from config
[2022-06-15 19:17:46,676] [DEBUG] Loading corpus from path: spacy_data/textcat_valid.spacy
[2022-06-15 19:17:46,677] [DEBUG] Loading corpus from path: spacy_data/textcat_train.spacy
[2022-06-15 19:17:46,678] [INFO] Pipeline: ['textcat_multilabel']
[2022-06-15 19:17:46,680] [INFO] Created vocabulary
[2022-06-15 19:17:46,681] [INFO] Finished initializing nlp object
[2022-06-15 19:17:57,346] [IN